In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
# For Data
from alpaca.data.timeframe import TimeFrame
from alpaca.data.historical import CryptoHistoricalDataClient, StockHistoricalDataClient

#from Data.historical_data import Historical_data
import pandas as pd
from Preprocessing.preprocessing import preprocessing_improved
from Models.LSTM.lightningLSTM import LightningLSTM
import torch
from torch.utils.data import Dataset, DataLoader
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger
from Preprocessing.dataclasses import StockPriceDataModule, StockDataset
import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, TQDMProgressBar, StochasticWeightAveraging
import matplotlib.pyplot as plt
import numpy as np 
import numpy as np
from tqdm import tqdm
from hyperopt import fmin, tpe, hp

In [4]:
df = pd.read_csv('Data\Stock\StockBars\MSFT_Minute')

In [ ]:
def objective(params):

    pbar.update(1)

    # Set the hyperparameters, batch size, and sequence length
    LEARNING_RATE = params['learning_rate']
    BATCH_SIZE = params['batch_size']
    SEQUENCE_LENGTH = params['sequence_length']
    
    data_train, data_test = preprocessing_improved(df = df[:int(len(df)*0.4)], lag = 1, sequence_length=SEQUENCE_LENGTH, dif_all=True, train_size=0.9 )

    # Create the data module with the given batch size and sequence length
    data_module = StockPriceDataModule(train_sequence = data_train,  test_sequence = data_test, batch_size=BATCH_SIZE, num_workers = 4)
    
    # Create the Lightning module with the given learning rate
    model = LightningLSTM(input_size=8, hidden_size = 125, num_layers = 1, learning_rate=LEARNING_RATE)
    
    # Train the model for a fixed number of epochs
    trainer = pl.Trainer(max_epochs=2, accelerator="gpu", devices=1, callbacks=[TQDMProgressBar(refresh_rate=100)])
    trainer.fit(model, data_module)
    
    # Evaluate the model on the test set and return the loss
    test_loss = trainer.test(model, datamodule=data_module)[0]['test_loss']
    
    # Log the performance for the current hyperparameter configuration
    log_dict = {
        'learning_rate': LEARNING_RATE,
        'batch_size': BATCH_SIZE,
        'sequence_length': SEQUENCE_LENGTH,
        'test_loss': test_loss,
    }
    with open('hyperparameter_logs.txt', 'a') as f:
        np.savetxt(f, np.array(list(log_dict.values())).reshape(1, -1), delimiter=',')
    
    return test_loss

space = {
    'learning_rate': hp.loguniform('learning_rate', -6, -1),
    'batch_size': hp.choice('batch_size', [16, 32, 64, 128]),
    'sequence_length': hp.choice('sequence_length', [32, 64, 128, 256])
}

# Set the maximum number of evaluations for the hyperparameter search
max_evals = 1

# Perform the hyperparameter search using the TPE algorithm
pbar = tqdm(total=max_evals)

best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=max_evals, rstate=np.random.default_rng (42))

pbar.close()

best